# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [46]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, loguniform
from azureml.core import Environment, ScriptRunConfig
import os
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
import logging
import csv
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.pipeline.steps import AutoMLStep
import joblib
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.core import Model

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [38]:
ws = Workspace.get(name = 'quick-starts-ws-260426',
                   subscription_id = '61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30',
                   resource_group = 'aml-quickstarts-260426')

# choose a name for experiment
experiment_name = 'heart-estimates'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-260426
Azure region: westus2
Subscription id: 61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30
Resource group: aml-quickstarts-260426


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [39]:
cluster_name = "notebook260426"
project_folder = './capstone'

found = False
description_text = "Heart Data for Udacity Capstone"

dataset = Dataset.get_by_name(ws, name='heart-data')

try:
    aml_compute = ComputeTarget(workspace=ws, name = cluster_name)
    print("Found existing cluster, use it")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes= 4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)

# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'AUC_weighted',
    "n_cross_validations": 5
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=cluster_name,
                             task = "classification",
                             training_data=dataset,
                             #time_column_name="date", 
                             label_column_name="DEATH_EVENT",  
                             #path = project_folder,
                             #enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

Found existing cluster, use it


In [40]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
heart-estimates,AutoML_4506947a-35c6-49f0-b99b-37268eeba8fe,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [41]:
# remote_run = experiment.submit(automl_config, show_output = False)

# RunDetails(remote_run).show()

# automl_run = experiment.start_logging()

# automl_run.log(name="message", value = "Hello from run!")

# print(automl_run.get_status())

RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [43]:
best_run_metrics_automl = remote_run.get_metrics()
print(best_run_metrics_automl["AUC_weighted"])

0.919685977297896


In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [52]:
model = remote_run.register_model(model_name = "automl-heart-sdk", metric="AUC_weighted")

TODO: In the cell below, send a request to the web service you deployed to test it.

In [45]:
# experiment_name = 'heart-capstone'
# project_folder = './heart-captstone'

# experiment = Experiment(ws, experiment_name)

# from azureml.pipeline.core import PipelineRun

# run_id = "AutoML_4506947a-35c6-49f0-b99b-37268eeba8fe_41"
# pipeline_run = PipelineRun(experiment, run_id)

# published_pipeline = pipeline_run.publish_pipeline(
#     name="Heart Capstone", description="Heart Capstone Pipeline", version="1.0")

# published_pipeline

ErrorResponseException: (NotFound) Response status code does not indicate success: 404 (Not Found).
Microsoft.RelInfra.Common.Exceptions.ErrorResponseException: GetExperimentEntity, input id is AutoML_4506947a-35c6-49f0-b99b-37268eeba8fe_41 experimentIds: plrv2id-UTC1W_gENWeE3wmiAioqxkCAx4_53i3uJAXudavrjro, plrid-dn8ytPP8D5JJyH-orPEHt1MG-MP1L14OFs8I4QDgAao, and all id retrieve entity not found.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
